In [57]:
import copy
import sys
sys.path.append('/home/kai/DAST/network')
import torch
import torch.nn as nn
import numpy as np
import numpy as np
import matplotlib.pyplot as plt
import os
import matplotlib.pyplot as plt
import sys
import os
import time
from torch.autograd import Variable
from DAST_utils import *
from DAST_Network import *
from torch.utils.data import TensorDataset, DataLoader
from tqdm import tqdm
from typing import List
os.environ["CUDA_VISIBLE_DEVICES"] = "0"
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)


In [58]:
class DASTModel():
    def __init__(self, train_datasets: List[str], test_dataset: List[str], data_path: str, is_norm: bool, hyper_parameters: dict, model_save_path: str, model_save_name: str) -> None:
        self.TRAIN_DATASETS = train_datasets
        self.TEST_DATASETS = test_dataset
        self.DATA_PATH = data_path
        self.MODEL_SAVE_PATH = model_save_path
        self.MODEL_SAVE_NAME = model_save_name
        self.X_train = []
        self.X_test = []
        self.Y_train = []
        self.Y_test = []
        self.HP = hyper_parameters
        self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
        self.best_predict = []
        self.last_predict_y = None
        self.train_loss_list = []
        self.test_loss_list = []
        self.best_mse_loss = 10000.0
        self.best_rmse_loss = None
        self.best_train_loss = 10000.0
        self.norm = '_norm' if is_norm else ''
        self.best_model_params = None
        
    @staticmethod
    def RMSE(target, pred):
        square_error = (target - pred) ** 2
        mse =  (torch.sum(square_error)) / len(target)
        rmse = mse ** 0.5
        return rmse
    
    @staticmethod
    def MAE(target, pred):
        absolute_error = np.abs(target - pred)
        return torch.sum(absolute_error) / len(target)
    
    def _load_x_y(self, folder: str):
        y_tmp = np.load(f'{self.DATA_PATH}/{folder}/{folder}_Y.npy')
        feature1 = np.load(f'{self.DATA_PATH}/{folder}/{folder}_X{self.norm}_2560.npy')
        feature2 = np.load(f'{self.DATA_PATH}/{folder}/{folder}_X{self.norm}_1280.npy')
        feature3 = np.load(f'{self.DATA_PATH}/{folder}/{folder}_X{self.norm}_640.npy')
        X_train = np.concatenate((feature1, feature2, feature3), axis=2)
        return X_train, np.reshape(y_tmp, ((len(y_tmp), -1)))
    
    def _concate(self):
        self.X_train = np.concatenate(self.X_train, axis=0)
        self.Y_train = np.concatenate(self.Y_train, axis=0)
        self.X_test = np.concatenate(self.X_test, axis=0)
        self.Y_test = np.concatenate(self.Y_test, axis=0)

    def _load_np(self,):
        # train
        for folder in self.TRAIN_DATASETS:
            X_train, Y_train = self._load_x_y(folder)
            self.X_train.append(X_train)
            self.Y_train.append(Y_train)
        # test
        for folder in self.TEST_DATASETS:
            X_test, Y_test = self._load_x_y(folder)
            self.X_test.append(X_test)
            self.Y_test.append(Y_test)
        
    def _loop_feature(self, X, selected_indices):
        extracted_values_list = []
        for i in range(7):
            for num in selected_indices:
                extracted_values = X[:, :, num + 16 * i]
                extracted_values_list.append(extracted_values)
        result_array = np.stack(extracted_values_list, axis=-1)
        return result_array
    
    def _select_feature(self, selected_indices):
        for i in range(len(self.X_train)):
            self.X_train[i] = self._loop_feature(self.X_train[i], selected_indices)
        for i in range(len(self.X_test)):
            self.X_test[i] = self._loop_feature(self.X_test[i], selected_indices)
        
    def _tensorizing(self):
        self.X_train = Variable(torch.Tensor(self.X_train).float())
        self.Y_train = Variable(torch.Tensor(self.Y_train).float())
        self.X_test = Variable(torch.Tensor(self.X_test).float())
        self.Y_test = Variable(torch.Tensor(self.Y_test).float())
        
    def _get_dataloader(self):
        train_dataset = TensorDataset(self.X_train, self.Y_train)
        train_loader = DataLoader(dataset=train_dataset, batch_size=self.HP['batch_size'], shuffle=False)
        test_dataset = TensorDataset(self.X_test, self.Y_test)
        test_loader = DataLoader(dataset=test_dataset, batch_size=self.HP['batch_size'], shuffle=False)
        return train_loader, test_loader
    
    def _get_model(self):
        model = DAST(self.HP['dim_val_s'], self.HP['dim_attn_s'], self.HP['dim_val_t'], self.HP['dim_attn_t'], self.HP['dim_val'], self.HP['dim_attn'], self.HP['time_step'], self.HP['feature_len'], self.HP['dec_seq_len'], self.HP['output_sequence_length'], self.HP['n_decoder_layers'], self.HP['n_encoder_layers'], self.HP['n_heads'], self.HP['debug'])
        model = model.to(self.device)
        optimizer = torch.optim.Adam(model.parameters(), lr=self.HP['lr'])
        criterion = nn.MSELoss()
        return model, optimizer, criterion
        
    def inference(self, selected_indices, pretrain_model_path, pretrain_model_name, output_path):
        self._load_np()
        self._select_feature(selected_indices)
        self._concate()
        self._tensorizing()
        criterion = nn.MSELoss()
        model = DAST(self.HP['dim_val_s'], self.HP['dim_attn_s'], self.HP['dim_val_t'], self.HP['dim_attn_t'], self.HP['dim_val'], self.HP['dim_attn'], self.HP['time_step'], self.HP['feature_len'], self.HP['dec_seq_len'], self.HP['output_sequence_length'], self.HP['n_decoder_layers'], self.HP['n_encoder_layers'], self.HP['n_heads'], self.HP['debug'])
        model.load_state_dict(torch.load(f'{pretrain_model_path}/{pretrain_model_name}.pt'))
        model.to(self.device)
        _, test_loader = self._get_dataloader()
        model.eval()
        prediction_list = []
        with torch.no_grad():
            for _ ,(batch_x, _) in enumerate(test_loader):
                batch_X = batch_x.to(self.device)
                prediction = model(batch_X)
                prediction_list.append(prediction)
        out_batch_pre = torch.cat(prediction_list).detach().cpu()
        rmse_loss = self.RMSE(self.Y_test, out_batch_pre, )
        mae_loss = self.MAE(self.Y_test, out_batch_pre, )
        test_loss = criterion(out_batch_pre, self.Y_test)
        self.test_loss_list.append(test_loss)
        # print('rmse_loss = ', f'{rmse_loss.item():.7f}',
        #         'mae_loss = ', f'{mae_loss.item():.7f}',
        #         'mse_loss = ', f'{test_loss.item():.7f}')
        # np.save(f'{output_path}/{pretrain_model_name}_{self.TEST_DATASETS[0]}.npy', out_batch_pre)
        return rmse_loss, test_loss

In [59]:
DATA_PATH = '../../../data/10FEMTO/processed_data/'
MODEL_SAVE_NAME = 'Bearing3_pretrain_7'
IS_NORM = False
MODEL_SAVE_PATH = '../../../model'

In [60]:
selected_indices = [1, 3, 5, 7, 9, 10, 14]
# selected_indices = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]
FEATURE_LEN = len(selected_indices)
FEATURE_SIZE = 20
EMBEDD = 10
HYPER_PARAMETERS = {
    'batch_size': 256,
    'dim_val': FEATURE_SIZE,
    'dim_attn': EMBEDD,
    'dim_val_t': FEATURE_SIZE,
    'dim_attn_t': EMBEDD,
    'dim_val_s': FEATURE_SIZE,
    'dim_attn_s': EMBEDD,
    'n_heads': 4,
    'n_decoder_layers': 1,
    'n_encoder_layers': 2,
    'lr': 1e-3,
    'epochs': 100,
    'time_step': 40,
    # limit how many last input used, important!
    'dec_seq_len': 6,
    'output_sequence_length': 1,
    'feature_len': FEATURE_LEN,
    'debug': True
}

In [61]:
PRETRAIN_MODEL_PATH = '../../../model'
NP_OUTPUT = '../../../data/np/'

def test_loop():
    TRAIN_DATASETS = ['Bearing2_7']
    # TEST_DATASET = ['Bearing2_4']
    TEST_DATASET = ['Bearing1_3', 'Bearing1_4', 'Bearing1_5', 'Bearing1_6', 'Bearing1_7' ,\
    'Bearing2_3', 'Bearing2_4', 'Bearing2_5', 'Bearing2_6', 'Bearing2_7' ,\
    'Bearing3_3']
    DATASETS = ['Bearing1', 'Bearing2', 'Bearing3']
    
    for dataset in TEST_DATASET:
        test_num = dataset[7]
        FINETUNE_DATASET = dataset[:8]
        for DATASET in DATASETS:
            if DATASET[7] != test_num:
                PRETRAIN_DATASET = DATASET
                PRETRAIN_MODEL_NAME = f'{PRETRAIN_DATASET}_pretrain_{FINETUNE_DATASET}_finetune_7'
            else:
                PRETRAIN_MODEL_NAME = f'{FINETUNE_DATASET}_pretrain_7'
            dast_model = DASTModel(train_datasets=TRAIN_DATASETS, test_dataset=[dataset], data_path=DATA_PATH, is_norm=IS_NORM, hyper_parameters=HYPER_PARAMETERS, model_save_path=MODEL_SAVE_PATH, model_save_name=MODEL_SAVE_NAME)
            rmse, mse = dast_model.inference(selected_indices, PRETRAIN_MODEL_PATH, PRETRAIN_MODEL_NAME, NP_OUTPUT)
            if (DATASET == FINETUNE_DATASET):
                print(dataset, DATASET, 'None', f'{rmse.item():.7f}', f'{mse.item():.7f}')
            else:
                print(dataset, DATASET, FINETUNE_DATASET, f'{rmse.item():.7f}', f'{mse.item():.7f}')

In [62]:
test_loop()

Bearing1_3 Bearing1 None 0.0632105 0.0039956
Bearing1_3 Bearing2 Bearing1 0.1519653 0.0230935
Bearing1_3 Bearing3 Bearing1 0.0953454 0.0090908
Bearing1_4 Bearing1 None 0.2971727 0.0883116
Bearing1_4 Bearing2 Bearing1 0.2504566 0.0627285
Bearing1_4 Bearing3 Bearing1 0.1300722 0.0169188
Bearing1_5 Bearing1 None 0.2064061 0.0426035
Bearing1_5 Bearing2 Bearing1 0.2103634 0.0442527
Bearing1_5 Bearing3 Bearing1 0.2041798 0.0416894
Bearing1_6 Bearing1 None 0.2077157 0.0431458
Bearing1_6 Bearing2 Bearing1 0.2147992 0.0461387
Bearing1_6 Bearing3 Bearing1 0.2070792 0.0428818
Bearing1_7 Bearing1 None 0.1842623 0.0339526
Bearing1_7 Bearing2 Bearing1 0.1997703 0.0399082
Bearing1_7 Bearing3 Bearing1 0.2176494 0.0473713
Bearing2_3 Bearing1 Bearing2 0.2234273 0.0499197
Bearing2_3 Bearing2 None 0.2290070 0.0524442
Bearing2_3 Bearing3 Bearing2 0.2396014 0.0574088
Bearing2_4 Bearing1 Bearing2 0.2063725 0.0425896
Bearing2_4 Bearing2 None 0.2127511 0.0452630
Bearing2_4 Bearing3 Bearing2 0.2085942 0.0435116